# ChatGPTをAPIから利用するために

## 3-3 入出力の長さの制限や課金に影響する「トークン」

### Tokenizerとtiktokenの紹介

In [ ]:
!pip install tiktoken==0.5.2

In [ ]:
import tiktoken

text = "It’s easy to make something cool with LLMs, but very hard to make something production-ready with them."

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

In [ ]:
text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
tokens = encoding.encode(text)
print(len(tokens))

## 3-4 Chat Completions APIにふれる環境準備

### Google Colabのノートブック作成

In [ ]:
print("Hello World")

### OpenAIのAPIキーの準備

In [ ]:
# 2023年11月、Google ColabにAPIキーなどの機密情報を安全に管理する機能が追加されました。
# 画面左側の鍵のマークから「シークレット」を開いて、「OPENAI_API_KEY」という名前でOpenAIのAPIキーを保存してから、このセルを実行してください。
#
# 参考: https://twitter.com/GoogleColab/status/1719798406195867814

import os

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 3-5 Chat Completions APIをさわってみる

### OpenAIのライブラリ

In [ ]:
# 2023年11月、openaiパッケージのバージョン1.0がリリースされました。
# その際、以前のバージョンとは使用方法が大きく変更されました。
# 以後のコードはopenaiのバージョン1.10.0で動作するようにしてあります。

!pip install openai==1.10.0

### Chat Completions APIの呼び出し

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello! I'm John."}
  ]
)

print(response)

### 会話履歴を踏まえた応答を得る

In [ ]:
client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."},
        {"role": "assistant", "content": "Hello John! How can I assist you today?"},
        {"role": "user", "content": "Do you know my name?"}
    ]
)

### ストリーミングで応答を得る

In [ ]:
stream = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! I'm John."}
  ],
  stream=True
)

for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(content)

### （コラム）Completions API

In [ ]:
# Completions APIで使用可能だったモデル「text-davinci-003」は、2024年1月に廃止されました。
# 代替としてCompletions APIでは「gpt-3.5-turbo-instruct」を使うことになったため、ここでは「gpt-3.5-turbo-instruct」を使用しています。

from openai import OpenAI

client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Hello! I'm John."
)

print(response)

In [ ]:
prompt = """Human: Hello! I'm John.
AI: Nice to meet you, John!
Human: Do you know my name?
AI: """

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=prompt
)
print(response)

## 3-6 Function calling

### Function callingのサンプルコード

~[OpenAI の公式ドキュメント](https://platform.openai.com/docs/guides/gpt/function-calling) をもとに一部改変したコードです。~

大幅アップデートされたOpenAIの公式ドキュメントの、次のページのコードをもとに一部改変したコードです。

https://platform.openai.com/docs/guides/function-calling

In [ ]:
import json

def get_current_weather(location, unit="celsius"):
    weather_info = {
        "location": location,
        "temperature": "25",
        "unit": "celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. Tokyo",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]

In [ ]:
messages = [{"role": "user", "content": "What's the weather like in Tokyo?"}]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    # Function callingでは、functionsというパラメータは非推奨となり、代わりに「tools」というパラメータを使用するのが推奨になりました。
    tools=tools
)

print(response)

In [ ]:
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls

available_functions = {
    "get_current_weather": get_current_weather,
}

# tool_callsには複数のツールの呼び出しが含まれる可能性があるため、ループで順に処理します。
for tool_call in tool_calls:
    function_name = tool_call.function.name
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)

    function_response = fuction_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    print(function_response)

    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

In [ ]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
)

print(second_response)